#**Imports:**

In [ ]:
Feature Vector Extraction,T5-XL,LMPred_Embeddings(DS1)

In [ ]:
!pip install -q transformers
!pip install -q transformers sentencePiece

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from transformers import BertModel, BertTokenizer, XLNetModel, XLNetTokenizer, T5EncoderModel, T5Tokenizer
import re
import os
import requests
import gc
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [ ]:
import os

# اتصال Google Drive به محیط کاری
from google.colab import drive
drive.mount('/content/drive')

# مسیر پوشه جدید در Google Drive
folder_path = '/content/drive/MyDrive/my_project/LMPred_Embeddings'

# بررسی وجود پوشه و ایجاد آن در صورت عدم وجود
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created successfully.")
else:
    print(f"Folder '{folder_path}' already exists.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder '/content/drive/MyDrive/my_project/LMPred_Embeddings' already exists.


In [ ]:
import os
import numpy as np
import torch
import re
from transformers import BertTokenizer, BertModel, T5Tokenizer, T5EncoderModel, XLNetTokenizer, XLNetModel

class LM_EMBED:
    def __init__(self, language_model, max_len, rare_aa):
        self.lang_model = language_model
        self.max_len = max_len
        self.rare_aa = rare_aa

        # وارد کردن توکنایزر و مدل از Rostlab ProtTrans:
        self.tokenizer, self.model = self._load_model_and_tokenizer()

        # تنظیم دستگاه به GPU در صورت موجود بودن:
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = self.model.to(self.device).eval()

    def _load_model_and_tokenizer(self):
        model_map = {
            'BERT-BFD': ('Rostlab/prot_bert_bfd', BertTokenizer, BertModel),
            'BERT': ('Rostlab/prot_bert', BertTokenizer, BertModel),
            'T5-XL-BFD': ('Rostlab/prot_t5_xl_bfd', T5Tokenizer, T5EncoderModel),
            'T5-XL-UNI': ('Rostlab/prot_t5_xl_uniref50', T5Tokenizer, T5EncoderModel),
            'XLNET': ('Rostlab/prot_xlnet', XLNetTokenizer, XLNetModel)
        }

        tokenizer_name, tokenizer_class, model_class = model_map.get(self.lang_model, (None, None, None))
        if tokenizer_name is None or tokenizer_class is None or model_class is None:
            raise ValueError("مدل زبانی پشتیبانی نمی‌شود")

        tokenizer = tokenizer_class.from_pretrained(tokenizer_name, do_lower_case=False)
        model = model_class.from_pretrained(tokenizer_name)

        # برای XLNet، تنظیم mem_len
        if self.lang_model == 'XLNET':
            model.config.mem_len = 512

        return tokenizer, model

    def add_spaces(self, seqs_list):
        return [" ".join(sequence) for sequence in seqs_list]

    def extract_word_embs(self, seq_df, filename):
        # ساختن لیستی از توالی‌ها از دیتافریم:
        seqs_list = seq_df.Sequence.to_list()
        seqs_spaced = self.add_spaces(seqs_list)

        if self.rare_aa:
            seqs_spaced = [re.sub(r"[UZOB]", "X", sequence) for sequence in seqs_spaced]

        ids = self.tokenizer.batch_encode_plus(seqs_spaced, add_special_tokens=True, padding='max_length', max_length=self.max_len)
        input_ids = torch.tensor(ids['input_ids']).to(self.device)
        attention_mask = torch.tensor(ids['attention_mask']).to(self.device)

        # پردازش توالی‌ها به صورت دسته‌ای
        batch_size = 10
        num_batches = (len(input_ids) + batch_size - 1) // batch_size
        all_embeddings = []

        with torch.no_grad():
            for i in range(num_batches):
                start = i * batch_size
                end = min((i + 1) * batch_size, len(input_ids))
                batch_input_ids = input_ids[start:end]
                batch_attention_mask = attention_mask[start:end]

                embeddings = self.model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)[0]
                all_embeddings.append(embeddings.cpu().numpy())

        embedding_res = np.concatenate(all_embeddings, axis=0)
        features = self.extract_features(embedding_res, attention_mask.cpu().numpy())
        padded_arr = self.pad(features)

        # مسیر ذخیره‌سازی امبدینگ
        embedding_folder = "/content/drive/MyDrive/my_project/LMPred_Embeddings"
        os.makedirs(embedding_folder, exist_ok=True)
        full_path = os.path.join(embedding_folder, filename)

        # ذخیره آرایه:
        print("در حال ذخیره امبدینگ‌ها...")
        np.save(full_path, padded_arr)

    def extract_features(self, emb_res, att_msk):
        features = []
        for seq_num in range(len(emb_res)):
            seq_len = (att_msk[seq_num] == 1).sum()
            if self.lang_model in ['BERT-BFD', 'BERT']:
                seq_emd = emb_res[seq_num][1:seq_len-1]
            elif self.lang_model in ['T5-XL-BFD', 'T5-XL-UNI']:
                seq_emd = emb_res[seq_num][:seq_len-1]
            elif self.lang_model == 'XLNET':
                padded_seq_len = len(att_msk[seq_num])
                seq_emd = emb_res[seq_num][padded_seq_len-seq_len:padded_seq_len-2]
            features.append(seq_emd)

        return np.array(features, dtype=object)

    def pad(self, features):
        dim1 = self.max_len - 2  # کاهش دو واحد برای توکن‌های CLS و SEP که از قبل حذف شده‌اند
        dim2 = features[0].shape[1]
        padded_arr = np.zeros((len(features), dim1, dim2))

        for i, feature in enumerate(features):
            padded_arr[i, :feature.shape[0], :feature.shape[1]] = feature

        return padded_arr


In [ ]:
X_val = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_AMP_Prediction/LM_Pred_Dataset/X_val_final.csv')


In [ ]:
X_test = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_AMP_Prediction/X_test_final.csv')


In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_AMP_Prediction/X_train_final.csv')


In [ ]:
# Specifying the max sequence length in the given dataset (255 for the LMPred Dataset),
# then adding 2 to account for special [CLS, SEP] tokens added by the language models):
max_seq_len = 257
T5XL_UNI_EMBED = LM_EMBED('T5-XL-UNI', max_seq_len, True)

In [ ]:
# تعریف مسیر خروجی برای ذخیره امبدینگ‌های X_val

output_val = '/content/drive/MyDrive/my_project/LMPred_Embeddings/T5XL_UNI_INDEP_X_val.npy'

# استخراج و ذخیره امبدینگ‌های X_val
T5XL_UNI_EMBED.extract_word_embs(X_val, output_val)

print("امبدینگ‌های X_val با موفقیت استخراج و ذخیره شدند.")


# استخراج و ذخیره امبدینگ‌های X_val
T5XL_UNI_EMBED.extract_word_embs(X_val, output_val)

print("امبدینگ‌های X_val با موفقیت استخراج و ذخیره شدند.")


در حال ذخیره امبدینگ‌ها...
امبدینگ‌های X_val با موفقیت استخراج و ذخیره شدند.


In [ ]:
import pandas as pd
import numpy as np

# بارگذاری داده‌ها از CSV
X_train = pd.read_csv('/content/drive/MyDrive/my_project/LMPred_AMP_Prediction/LM_Pred_Dataset/X_train_final.csv')

# پارامترهای مربوط به دسته‌های داده
batch_size = 1000  # تعداد نمونه‌ها در هر دسته
num_batches = (len(X_train) + batch_size - 1) // batch_size  # تعداد کل دسته‌ها

# مسیر خروجی برای ذخیره امبدینگ‌ها
output_dir = '/content/drive/MyDrive/my_project/LMPred_Embeddings/'

# اطمینان از اینکه آبجکت T5XL_UNI_EMBED به درستی ایجاد شده است
if 'T5XL_UNI_EMBED' not in locals():
    raise ValueError("آبجکت T5XL_UNI_EMBED تعریف نشده است.")

# پردازش دسته به دسته
for i in range(num_batches):
    # تعریف محدوده داده‌های فعلی
    start_index = i * batch_size
    end_index = min(start_index + batch_size, len(X_train))

    # استخراج داده‌های فعلی
    X_train_batch = X_train.iloc[start_index:end_index]

    # مسیر خروجی برای ذخیره امبدینگ‌های دسته فعلی
    output_batch = f'{output_dir}T5XL_UNI_INDEP_X_TRAIN_batch_{i+1}.npy'

    # استخراج و ذخیره امبدینگ‌های دسته فعلی
    T5XL_UNI_EMBED.extract_word_embs(X_train_batch, output_batch)

    print(f"امبدینگ‌های دسته {i+1} با موفقیت استخراج و ذخیره شدند.")

print("تمام امبدینگ‌ها با موفقیت استخراج و ذخیره شدند.")


در حال ذخیره امبدینگ‌ها...
امبدینگ‌های دسته 1 با موفقیت استخراج و ذخیره شدند.
در حال ذخیره امبدینگ‌ها...
امبدینگ‌های دسته 2 با موفقیت استخراج و ذخیره شدند.
در حال ذخیره امبدینگ‌ها...
امبدینگ‌های دسته 3 با موفقیت استخراج و ذخیره شدند.
در حال ذخیره امبدینگ‌ها...
امبدینگ‌های دسته 4 با موفقیت استخراج و ذخیره شدند.
در حال ذخیره امبدینگ‌ها...
امبدینگ‌های دسته 5 با موفقیت استخراج و ذخیره شدند.
در حال ذخیره امبدینگ‌ها...
امبدینگ‌های دسته 6 با موفقیت استخراج و ذخیره شدند.
تمام امبدینگ‌ها با موفقیت استخراج و ذخیره شدند.


In [ ]:
# تعریف مسیر خروجی برای ذخیره امبدینگ‌های X_test

output_test = '/content/drive/MyDrive/my_project/LMPred_Embeddings/T5XL_UNI_INDEP_X_TEST.npy'

# استخراج و ذخیره امبدینگ‌های X_test
T5XL_UNI_EMBED.extract_word_embs(X_test, output_test)

print("امبدینگ‌های X_test با موفقیت استخراج و ذخیره شدند.")


# استخراج و ذخیره امبدینگ‌های X_test
T5XL_UNI_EMBED.extract_word_embs(X_test, output_test)

print("امبدینگ‌های X_test با موفقیت استخراج و ذخیره شدند.")


در حال ذخیره امبدینگ‌ها...
امبدینگ‌های X_test با موفقیت استخراج و ذخیره شدند.
در حال ذخیره امبدینگ‌ها...
امبدینگ‌های X_test با موفقیت استخراج و ذخیره شدند.
